In [77]:
from data_util import load_embeddings, DMConfig, tfConfig
try:
    import _pickle as cPickle
except ImportError:
    import cPickle
import gzip
import os
import numpy as np
from defs import LBLS

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
config = DMConfig()
if os.path.exists(config.tok2id_path):
    with gzip.open(os.path.abspath(config.tok2id_path)) as f:
        tok2id = cPickle.load(f)
if os.path.exists(config.id2tok_path):
    with gzip.open(os.path.abspath(config.id2tok_path)) as f:
        id2tok = cPickle.load(f)

In [18]:
embeddings = load_embeddings(config, tok2id)

INFO:Initialized embeddings.


In [19]:
embeddings.shape

(10008, 50)

## Training

In [20]:
from tf_linear_classifier import TfLinearClassifier
from defs import PROJECT_DIR

In [21]:
model = TfLinearClassifier(embedding=embeddings, 
                           ckpts_prefix='{}/checkpoints/linear_model/linear-model'.format(PROJECT_DIR)
                          )
model.fit(tfConfig, restore_weights=True, batches_to_eval=50, max_iter=0)

INFO:Initializing data manager...
INFO:took 0 s
INFO:Building model...
INFO:took 0 s


INFO:tensorflow:Restoring parameters from /Users/tuckerleavitt/Documents/CS224U/project/checkpoints/linear_model/linear-model-3550


INFO:Restoring parameters from /Users/tuckerleavitt/Documents/CS224U/project/checkpoints/linear_model/linear-model-3550
INFO:-- Restored model


## Evaluation

In [92]:
def print_example(pred, label, feats, length, email_id):
    ids_ = feats[:length][:, 0]
    print("==== {} ====".format(email_id))
    print("Predicted label: {}. True label: {}".format(LBLS[pred], LBLS[label]))
    print( " ".join((id2tok[i] for i in ids_)) )
    print("------------------------------------")

    

def print_examples(preds, outputs, inputs, lens, email_ids):
    for p, o, i, l, d in zip(preds, outputs, inputs, lens, email_ids):
        print_example(p, o, i, l, d)

In [93]:
preds, inputs, lens, outputs, email_ids = model.predict()
correct = preds == outputs
print("Predicted {} of {} examples correctly (accuracy: {:.3f})".format(np.sum(correct), correct.shape[0], np.mean(correct)))

Predicted 29 of 32 examples correctly (accuracy: 0.906)


In [94]:
print_examples(preds, outputs, inputs, lens, email_ids)

==== b'<24049886.1075855776034.JavaMail.evans@thyme>' ====
Predicted label: NORESPONSE. True label: NORESPONSE
<s> as you know , technology is critical to enron ; it drives our success and wil= l=20 continue to do so in the future . technology has helped enron create new=20 businesses like enron broadband services and enron net works , and it is=20 responsible for applications such as enrononline and enroncredit.com . you= =01 , ve=20 seen what technology can do at work . now we want you and your family to=20 realize its benefits at home . with that in mind , we are excited to let you know that we are introducing t= he=20 clickathome program , which will give each employee a computer for use at home . UUUNKKK technology permits , we will also subsidize an internet connection . with the= =20 click of a mouse , a home computer plus internet access will put a world of= =20 internet knowledge at your UUUNKKK , s UUUNKKK . =20 we have just signed an agreement with dell computer corporation 